In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import os

# Đường dẫn đến mô hình tuổi (cập nhật đường dẫn này)
AGE_MODEL_PATH = "checkpoints/age/model_final.pth"

# Định nghĩa mô hình AgeRegressor
class AgeRegressor(torch.nn.Module):
    def __init__(self):
        super(AgeRegressor, self).__init__()
        # CNN đơn giản cho hồi quy
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(3, 16, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Conv2d(16, 32, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2)
        )
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(32 * 56 * 56, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 1)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.regressor(x)
        return x

# Mô hình giả khi không tải được mô hình thật
class MockModel:
    def __init__(self):
        pass
        
    def __call__(self, x):
        # Trả về tuổi giả mặc định
        return torch.tensor([45.0])
        
    def eval(self):
        return self

# Hàm biến đổi hình ảnh
def get_transform():
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

# Hàm tải mô hình tuổi
def load_age_model():
    try:
        age_state_dict = torch.load(AGE_MODEL_PATH, map_location=torch.device('cpu'))
        age_model = AgeRegressor()
        
        # Xử lý tiền tố "module." nếu có trong state dict
        if all(key.startswith("module.") for key in age_state_dict.keys()):
            age_state_dict = {k[7:]: v for k, v in age_state_dict.items()}
            
        age_model.load_state_dict(age_state_dict, strict=False)
        age_model.eval()
        print("Mô hình tuổi đã được tải thành công.")
    except Exception as e:
        print(f"Lỗi khi tải mô hình tuổi: {str(e)}")
        age_model = MockModel()
        print("Sử dụng mô hình giả cho dự đoán tuổi.")
    
    return age_model

# Hàm dự đoán tuổi
def predict_age(image, age_model):
    transform = get_transform()
    image_tensor = transform(image).unsqueeze(0)
    
    with torch.no_grad():
        try:
            age_output = age_model(image_tensor)
            predicted_age = age_output.item()
        except Exception as e:
            print(f"Lỗi trong dự đoán tuổi: {e}")
            predicted_age = 45.0  # Giá trị mặc định khi lỗi
    
    return round(predicted_age)

# Phần chính để chạy ví dụ
if __name__ == "__main__":
    # Tải mô hình
    age_model = load_age_model()
    
    # Đường dẫn đến hình ảnh mẫu (cập nhật đường dẫn này)
    sample_image_path = "path/to/sample_image.jpg"
    
    if os.path.exists(sample_image_path):
        sample_image = Image.open(sample_image_path).convert('RGB')
        
        # Dự đoán tuổi
        predicted_age = predict_age(sample_image, age_model)
        print(f"Tuổi dự đoán: {predicted_age} ngày")
    else:
        print(f"Hình ảnh không tìm thấy tại: {sample_image_path}")